## Shomate equation
The Shomate equation is used to estimate standart formation enthalpy, entropy and specific heat from fitted parameters commonly available in several databases such as [NIST](https://webbook.nist.gov/cgi/cbook.cgi?ID=C7732185&Type=JANAFL&Table=on). The NIST format is:

$$
C_p^\circ = A + Bt + Ct^2 + Dt^3 + E/t^2\\
H^\circ - H^\circ_{298.15K}= At + Bt^2/2 + Ct^3/3 + Dt^4/4 - E/t + F -H\\
S^\circ = A\ln{t} + Bt + Ct^2/2 + Dt^3/3 -E/(2t^2) + G
$$

with $A$, $B$, $C$, $D$ and $E$ constants, $C_p$ heat capacity in $J/mol K$, $H^\circ$ the standard enthapy in $kJ/mol$, $S^\circ$ standard entropy in $J/molK$ and $t$ as the temperature in $K$ over $1000$ ($t=$temp.$/1000$).

The shomate equation is included in the stego package taking as input a dictionary with the constants and standard values of $H^\circ$ and $S^\circ$.

In [1]:
import stegoplot as stg

ModuleNotFoundError: No module named 'stegoplot'